In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/defense_evasion/host/covenant_lolbin_wuauclt_createremotethread.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','Image','CommandLine']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 1)
    & (df['Image'].str.lower().str.endswith('wuauclt.exe', na=False))
    & (df['CommandLine'].str.lower().str.contains('.*wuauclt.*updatedeploymentprovider.*.dll.*runhandlercomserver.*', regex=True))
]
.head()
)

,@timestamp,Hostname,Image,CommandLine
593,2020-10-12T22:34:38.928Z,WORKSTATION5.theshire.local,C:\Windows\System32\wuauclt.exe,C:\Windows\System32\wuauclt.exe /UpdateDeploy...


In [4]:
(
df[['@timestamp','Hostname','Image','ImageLoaded']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['Image'].str.lower().str.endswith('wuauclt.exe', na=False))
    & (df['Signed'] == 'false')
]
.head()
)

,@timestamp,Hostname,Image,ImageLoaded
612,2020-10-12T22:34:38.938Z,WORKSTATION5.theshire.local,C:\Windows\System32\wuauclt.exe,C:\ProgramData\SimpleInjection.dll


In [5]:
(
df[['@timestamp','Hostname','SourceImage','TargetImage']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 8)
    & (df['SourceImage'].str.lower().str.endswith('wuauclt.exe', na=False))
]
.head()
)

,@timestamp,Hostname,SourceImage,TargetImage
613,2020-10-12T22:34:38.939Z,WORKSTATION5.theshire.local,C:\Windows\System32\wuauclt.exe,&lt;unknown process&gt;


In [13]:
imageLoadDf = (
# (
df[['@timestamp','Hostname','Image','ImageLoaded']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['Image'].str.lower().str.endswith('wuauclt.exe', na=False))
]
)

fileCreateDf = (
# (
df[['@timestamp','Hostname','Image','TargetFilename']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 11)
]
)

(
pd.merge(imageLoadDf, fileCreateDf,
    left_on = 'ImageLoaded', right_on = 'TargetFilename', how = 'inner')
)

,@timestamp_x,Hostname_x,Image_x,ImageLoaded,@timestamp_y,Hostname_y,Image_y,TargetFilename
0,2020-10-12T22:34:38.938Z,WORKSTATION5.theshire.local,C:\Windows\System32\wuauclt.exe,C:\ProgramData\SimpleInjection.dll,2020-10-12T22:33:14.142Z,WORKSTATION5.theshire.local,C:\Users\pgustavo\Desktop\GruntHTTP.exe,C:\ProgramData\SimpleInjection.dll


In [20]:
imageLoadDf = (
# (
df[['@timestamp','Hostname','Image','ImageLoaded','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 7)
    & (df['Image'].str.lower().str.endswith('wuauclt.exe', na=False))
]
)

fileCreateDf = (
# (
df[['@timestamp','Hostname','Image','TargetFilename','ProcessGuid']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 11)
]
)

firstJoinDf = (
pd.merge(imageLoadDf, fileCreateDf,
    left_on = 'ImageLoaded', right_on = 'TargetFilename', how = 'inner')
)

createRTDf = (
# (
df[['@timestamp','Hostname','SourceImage','SourceProcessGuid','TargetImage']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 8)
    & (df['SourceImage'].str.lower().str.endswith('wuauclt.exe', na=False))
]
)

(
pd.merge(firstJoinDf, createRTDf,
    left_on = 'ProcessGuid_x', right_on = 'SourceProcessGuid', how = 'inner')
)

,@timestamp_x,Hostname_x,Image_x,ImageLoaded,ProcessGuid_x,@timestamp_y,Hostname_y,Image_y,TargetFilename,ProcessGuid_y,@timestamp,Hostname,SourceImage,SourceProcessGuid,TargetImage
0,2020-10-12T22:34:38.938Z,WORKSTATION5.theshire.local,C:\Windows\System32\wuauclt.exe,C:\ProgramData\SimpleInjection.dll,{71b87c12-d9fe-5f84-1901-000000000500},2020-10-12T22:33:14.142Z,WORKSTATION5.theshire.local,C:\Users\pgustavo\Desktop\GruntHTTP.exe,C:\ProgramData\SimpleInjection.dll,{71b87c12-d7e5-5f84-0d01-000000000500},2020-10-12T22:34:38.939Z,WORKSTATION5.theshire.local,C:\Windows\System32\wuauclt.exe,{71b87c12-d9fe-5f84-1901-000000000500},&lt;unknown process&gt;
